In [1]:
import pandas as pd

In [2]:
hits = pd.read_csv("../data/hits.csv")
noise = pd.read_csv("../data/noise.csv")

# Manual inspection
Make sure the distribution before and after merging the dataframes is more or less the same.

In [4]:
print("hits shape: {0}, noise shape: {1}, total shape: {2}".format(hits.shape, noise.shape, hits.shape[0]+noise.shape[0]))

hits shape: (489906, 11), noise shape: (45330980, 11), total shape: 45820886


In [16]:
noise.describe()

,time,dom_id,pmt_id,tot,pos_x,pos_y,pos_z,dir_x,dir_y,dir_z,label
count,4.533098e+07,4.533098e+07,4.533098e+07,4.533098e+07,4.533098e+07,4.533098e+07,4.533098e+07,4.533098e+07,4.533098e+07,4.533098e+07,45330980.0
mean,5.006286e+07,1.035591e+03,1.501094e+01,2.643602e+01,8.150053e-03,2.416813e-04,1.171760e+02,1.938109e-04,-3.343916e-05,-1.951633e-01,0.0
std,2.890988e+07,5.975733e+02,8.942334e+00,2.458031e+00,5.125542e+01,6.222462e+01,4.866544e+01,5.809644e-01,5.810222e-01,5.354744e-01,0.0
min,-2.000000e+00,1.000000e+00,0.000000e+00,1.200000e+01,-9.462700e+01,-1.156000e+02,3.770000e+01,-9.550000e-01,-9.550000e-01,-1.000000e+00,0.0
25%,2.502273e+07,5.180000e+02,7.000000e+00,2.500000e+01,-4.501800e+01,-5.792500e+01,7.404100e+01,-4.780000e-01,-4.780000e-01,-5.560000e-01,0.0
50%,5.006429e+07,1.036000e+03,1.500000e+01,2.600000e+01,1.309000e+00,-4.184000e+00,1.217000e+02,0.000000e+00,-0.000000e+00,-2.960000e-01,0.0
75%,7.509979e+07,1.553000e+03,2.300000e+01,2.800000e+01,4.045200e+01,4.854100e+01,1.602410e+02,4.780000e-01,4.780000e-01,2.960000e-01,0.0
max,1.015914e+08,2.070000e+03,3.000000e+01,4.000000e+01,9.624300e+01,1.050240e+02,1.966110e+02,9.550000e-01,9.550000e-01,5.580000e-01,0.0


In [17]:
hits.describe()

,dom_id,pmt_id,pos_x,pos_y,pos_z,dir_x,dir_y,dir_z,tot,time,label
count,489906.000000,489906.000000,489906.000000,489906.000000,489906.000000,489906.000000,489906.000000,489906.000000,489906.000000,4.899060e+05,489906.0
mean,1048.735653,32496.026419,0.341670,-1.514543,119.205209,0.000912,-0.002101,-0.182805,26.056821,4.893892e+07,1.0
std,592.666757,18372.661240,50.964521,62.401296,47.553276,0.582604,0.583274,0.535609,9.273521,2.905600e+07,0.0
min,1.000000,1.000000,-94.627000,-115.600000,37.700000,-0.955000,-0.955000,-1.000000,1.000000,2.844900e+04,1.0
25%,537.000000,16622.000000,-44.772000,-58.113000,74.211000,-0.478000,-0.478000,-0.556000,23.000000,2.393785e+07,1.0
50%,1056.000000,32727.000000,1.424000,-4.581000,121.789000,0.000000,-0.000000,-0.296000,26.000000,4.810140e+07,1.0
75%,1562.000000,48398.750000,40.478000,48.480000,160.241000,0.478000,0.478000,0.296000,28.000000,7.434630e+07,1.0
max,2070.000000,64170.000000,96.243000,105.024000,196.611000,0.955000,0.955000,0.558000,209.000000,9.998601e+07,1.0


In [3]:
df = pd.concat([hits, noise])
print("df shape: {0}".format(df.shape))

df shape: (45820886, 11)


In [4]:
df.isna().any().any()

False

In [15]:
df.describe()

,dom_id,pmt_id,pos_x,pos_y,pos_z,dir_x,dir_y,dir_z,tot,time,label
count,4.582089e+07,4.582089e+07,4.582089e+07,4.582089e+07,4.582089e+07,4.582089e+07,4.582089e+07,4.582089e+07,4.582089e+07,4.582089e+07,4.582089e+07
mean,1.035732e+03,3.622902e+02,1.171597e-02,-1.595403e-02,1.171977e+02,2.014941e-04,-5.554718e-05,-1.950312e-01,2.643196e+01,5.005085e+07,1.069176e-02
std,5.975226e+02,3.842982e+03,5.125233e+01,6.222670e+01,4.865414e+01,5.809820e-01,5.810463e-01,5.354774e-01,2.626463e+00,2.891168e+07,1.028467e-01
min,1.000000e+00,0.000000e+00,-9.462700e+01,-1.156000e+02,3.770000e+01,-9.550000e-01,-9.550000e-01,-1.000000e+00,1.000000e+00,-2.000000e+00,0.000000e+00
25%,5.180000e+02,7.000000e+00,-4.501800e+01,-5.792500e+01,7.404100e+01,-4.780000e-01,-4.780000e-01,-5.560000e-01,2.500000e+01,2.500360e+07,0.000000e+00
50%,1.036000e+03,1.500000e+01,1.309000e+00,-4.184000e+00,1.217000e+02,0.000000e+00,0.000000e+00,-2.960000e-01,2.600000e+01,5.004222e+07,0.000000e+00
75%,1.553000e+03,2.300000e+01,4.045200e+01,4.854100e+01,1.602410e+02,4.780000e-01,4.780000e-01,2.960000e-01,2.800000e+01,7.509190e+07,0.000000e+00
max,2.070000e+03,6.417000e+04,9.624300e+01,1.050240e+02,1.966110e+02,9.550000e-01,9.550000e-01,5.580000e-01,2.090000e+02,1.015914e+08,1.000000e+00


# Drop rows with negative time

In [5]:
df = df[df.time >= 0.0]
df = df.sort_values(by=['time'])
df.shape

(45820216, 11)

In [6]:
df.label.unique()

array([0, 1])

In [11]:
len(df.pmt_id.unique())

64114

In [7]:
df

,dom_id,pmt_id,pos_x,pos_y,pos_z,dir_x,dir_y,dir_z,tot,time,label
7011482,321.0,4.0,-17.661,32.245,65.231,-0.460,-0.266,-0.847,26.0,0.0,0
36188506,1653.0,22.0,11.595,85.465,65.459,-0.955,-0.000,0.296,27.0,0.0,0
6008280,275.0,8.0,-36.464,67.166,160.189,0.415,0.720,-0.556,26.0,0.0,0
36341659,1660.0,22.0,61.660,101.635,169.059,-0.955,-0.000,0.296,26.0,0.0,0
21139713,966.0,15.0,-54.510,-78.323,94.341,-0.827,0.478,-0.296,24.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
11438019,523.0,12.0,-57.230,-5.401,196.389,0.415,-0.720,-0.556,28.0,101502104.0,0
8460308,387.0,11.0,0.724,66.341,121.789,-0.415,-0.720,-0.556,24.0,101516467.0,0
2062008,95.0,3.0,-26.436,86.737,160.131,-0.460,0.266,-0.847,23.0,101545421.0,0
16221364,741.0,29.0,-26.931,-21.994,178.511,0.000,-0.830,0.558,27.0,101581891.0,0


In [19]:
df.to_csv("../data/data.csv")